## ZOANA BICH RESORT

#### Libraries

In [243]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [244]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd

In [245]:
#funcion de limpieza de columnas : 
#funcion de limpieza de columnas : 
import unicodedata
def limpiar_columnas(df):
    # Convertimos los nombres de columna a string
    df.columns = df.columns.astype(str)

    # Normalizamos las tildes usando unicodedata
    df.columns = [
        unicodedata.normalize('NFKD', col)
        .encode('ascii', 'ignore')
        .decode('utf-8')
        for col in df.columns
    ]

    # Aplicamos las transformaciones
    df.columns = (
        pd.Index(df.columns)
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace(r'[^\w\s]', '', regex=True)
    )
    
    return df


In [246]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Ejemplo de uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resources/ZUANA_BEACH_RESORT.pdf")

Analizando: resources/ZUANA_BEACH_RESORT.pdf
Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...
Extrayendo tablas...


In [247]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("ZUANA_BEACH_RESORT.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [248]:
datos_generales = {
    "Empresa":                re.search(r"^(C\.B\.\s*HOTELES\s+Y\s+RESORTS\s+S\.A\.)", textos, re.MULTILINE).group(1),
    "NIT Empresa":            re.search(r"Nit\.\s*(\d+)", textos).group(1),
    "Dirección Empresa":      re.search(r"Dirección:\s*(.+)", textos).group(1),
    "Teléfonos Empresa":      re.search(r"Teléfonos:\s*(.+)", textos).group(1),
    "Fecha Impresión":        re.search(r"Fecha Impresión:\s*\n(.+)", textos).group(1).strip(),
    "Hora Impresión":         re.search(r"Hora:\s*(.+)", textos).group(1).strip(),
    "Página":                 re.search(r"Página:\s*(\d+\s+de\s+\d+)", textos).group(1),
    "Fecha de creación":      re.search(r"Fecha de creación\s*:\s*([0-9/]+\s+[0-9:APMapm]+)", textos).group(1),
    "Ciudad":                 re.search(r"Ciudad:\s*(.+)", textos).group(1),
    "Consecutivo":            re.search(r"Consecutivo\s*:\s*(\d+)", textos).group(1),
    "Consecutivo BU":         re.search(r"Consecutivo BU\s*:\s*(\d+)", textos).group(1),
    "BU":                     re.search(r"BU\s*:\s*(.+)", textos).group(1),
    "Fecha Orden":            re.search(r"^Fecha\s*:\s*([0-9/]+)", textos, re.MULTILINE).group(1),
    "Fecha Entrega":          re.search(r"Fecha Entrega\s*:\s*([0-9/]+)", textos).group(1),
    "Días de Plazo":          re.search(r"Días de Plazo\s*:\s*(\d+)", textos).group(1),
    "Forma de Pago":          re.search(r"Forma de Pago\s*:\s*(.+)", textos).group(1),
    "Moneda":                 re.search(r"Moneda\s*:\s*(.+)", textos).group(1),
    "Requisición":            re.search(r"Requisición\s*:\s*(\d+)", textos).group(1),
    "Doc/Rel":                re.search(r"Doc/Rel\s*:\s*(.+)", textos).group(1),
    "Estado":                 re.search(r"Estado\s*:\s*(.+)", textos).group(1),
    "Detalles":               re.search(r"Detalles\s*:\s*(.+)", textos).group(1),
    "Observaciones":          re.search(r"Observaciones\s*:\s*(.+)", textos).group(1),
    "Proveedor":              re.search(r"Proveedor\s*:\s*(.+)", textos).group(1),
    "NIT Proveedor":          re.search(r"Proveedor\s*:[\s\S]*?Nit\s*:\s*(\d+)", textos).group(1),
    "Teléfono Proveedor":     re.search(r"Teléfono\s*:\s*([0-9]+)", textos).group(1),
    "Dirección Proveedor":    re.search(r"Dirección\s*:\s*(CL\s*\d+\s*\d+\s*\d+)", textos).group(1),
    "Ciudad Proveedor":       re.search(r"Ciudad:\s*(.+)", textos.split("Proveedor")[1]).group(1),
    "Total Items":            re.search(r"Total Items\s*[:\s]*([\d.,]+)", textos).group(1),
    "Total Descuento":        re.search(r"Total Descuento\s*:\s*([\d.,]+)", textos).group(1),
    "Sub-Total":              re.search(r"Sub-Total\s*:\s*([\d.,]+)", textos).group(1),
    "IVA":                    re.search(r"\bIVA\s*:\s*([\d.,]+)", textos).group(1),
    "Impuestos Aplicados":    re.search(r"Impuestos Aplicados\s*:\s*([\d.,]+)", textos).group(1),
    "Total Orden de Compra":  re.search(r"TOTAL ORDEN DE COMPRA\s*:\s*([\d.,]+)", textos).group(1),
}

In [249]:
texto = textos

In [250]:


# diccionario del encabezado
encabezado = {
    "Consecutivo": re.search(r"Consecutivo\s*:\s*(\d+)", texto).group(1),
    "Consecutivo BU": re.search(r"Consecutivo BU\s*:\s*(\d+)", texto).group(1),
    "BU": re.search(r"BU\s*:\s*(\w+)", texto).group(1),
    "Fecha": re.search(r"Fecha\s*:\s*([\d/]+)", texto).group(1),
    "Fecha Entrega": re.search(r"Fecha Entrega\s*:\s*([\d/]+)", texto).group(1),
    "Días de Plazo": int(re.search(r"Días de Plazo\s*:\s*(\d+)", texto).group(1)),
    "Forma de Pago": re.search(r"Forma de Pago\s*:\s*(.+)", texto).group(1).strip(),
    "Moneda": re.search(r"Moneda\s*:\s*(.+)", texto).group(1).strip(),
    "Requisición": re.search(r"Requisición\s*:\s*(\d+)", texto).group(1),
    "Documento Relacionado": re.search(r"Doc/Rel\s*:\s*(.+)", texto).group(1).strip(),
    "Estado": re.search(r"Estado\s*:\s*(.+)", texto).group(1).strip(),
    "Proveedor": re.search(r"Proveedor\s*:\s*(.+)", texto).group(1).strip(),
    "NIT Proveedor": re.search(r"Nit\s*:\s*(\d+)", texto).group(1),
    "Teléfono Proveedor": re.search(r"Teléfono\s*:\s*(\d+)", texto).group(1),
    "Dirección Proveedor": re.search(r"Dirección\s*:\s*(.+)", texto).group(1).strip(),
    "Ciudad Proveedor": re.search(r"Ciudad:\s*(.+)", texto).group(1).strip()
}

### Df general - Escoger Columnas -Ordenar dataframe
1. Cliente
2. Direccion
3. Destino
4. Numero de Pedido
5. Fecha de Orden
6. Fecha de Entrega
7. Id Falso (Para Cruce de Orden de compra )

In [251]:
df_encabezado = pd.DataFrame([encabezado])
df_encabezado  = limpiar_columnas(df_encabezado )


In [252]:

#Create a false column
df_encabezado['CLIENTE'] = 'ZUANA BEACH RESORT'
df_encabezado['DIRECCION'] = df_encabezado['direccion_proveedor']
df_encabezado['DESTINO'] = df_encabezado['direccion_proveedor']
df_encabezado['NUMERO_DE_PEDIDO'] = df_encabezado['consecutivo']
df_encabezado['FECHA_DE_ORDEN'] = df_encabezado['fecha']
df_encabezado['FECHA_DE_ENTREGA'] = df_encabezado['fecha_entrega']

#Create id False column
df_encabezado['id'] =  1

#Seleccionar Columnas 
df_encabezado = df_encabezado[['id', 'CLIENTE', 'DIRECCION', 'DESTINO', 'NUMERO_DE_PEDIDO',
                                'FECHA_DE_ORDEN', 'FECHA_DE_ENTREGA']]

#SUMAR 8 DIAS A LA FECHA_DE_ENTREGA
df_encabezado['FECHA_DE_ENTREGA'] = pd.to_datetime(df_encabezado['FECHA_DE_ENTREGA']) + pd.Timedelta(days=8)

#Castear a dia la fecha de la orden
df_encabezado['FECHA_DE_ORDEN'] = pd.to_datetime(df_encabezado['FECHA_DE_ORDEN']).dt.date

In [253]:
# Limpieza de columnas
df_encabezado

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA
0,1,ZUANA BEACH RESORT,CRA 2 NO. 6-80 BELLOHORIZONTE,CRA 2 NO. 6-80 BELLOHORIZONTE,72035,2025-08-03,2025-08-11


#### Patrones de Producto

In [254]:
patron_item = re.compile(
    r"(?P<codigo>\d{6})\s+(?P<nombre>[\w\s]+)\s+(?P<presentacion>\w+)\s+(?P<cantidad>\d+)\s+"
    r"(?P<valor_unitario>[\d.,]+)\s+(?P<dcto>[\d.]+)\s+(?P<iva>[\d.]+)\s+"
    r"(?P<total_ico>[\d.,]+)\s+(?P<v_unit>[\d.,]+)\s+(?P<ico>[\d.,]+)"
)

items = []
for match in patron_item.finditer(texto.replace(',', '')):
    item = match.groupdict()
    # Convertir strings a números
    item['cantidad'] = int(item['cantidad'])
    for campo in ['valor_unitario', 'dcto', 'iva', 'total_ico', 'v_unit', 'ico']:
        item[campo] = float(item[campo])
    items.append(item)

df_items = pd.DataFrame(items)

#### Escoger Columnas Productos 
1. Numero de item
2. descripcion
3. Cantidad
4. Unidad de medida
5. Costo Unitario
6. importe
7. id falso (Unir para realizar insersion de datos)

In [255]:
#Numero de Item

df_items['NUMERO_DE_ITEM'] = df_items['presentacion']
df_items['DESCRIPCION'] = df_items['nombre']
df_items['CANTIDAD'] = df_items['valor_unitario']
df_items['UNIDAD_DE_MEDIDA'] = 'UNIDAD'
df_items['COSTO_UNITARIO'] = df_items['ico']
df_items['IMPORTE'] = df_items['v_unit']

# Escoger Columnas
df_items = df_items[['NUMERO_DE_ITEM', 'DESCRIPCION', 'CANTIDAD', 'UNIDAD_DE_MEDIDA',
                     'COSTO_UNITARIO', 'IMPORTE']]

#Crear el id de la tabla
df_items['id'] = 1

In [256]:
df_items

,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE,id
0,PAQX35GR,DON CHICHARRON,192.0,UNIDAD,3166.19,607908.48,1
1,UNDX50GR,ACHIRAS,24.0,UNIDAD,3761.90,90285.60,1
2,PAQX100GR,DON CHICHARRON,100.0,UNIDAD,8565.00,856500.00,1


#### Join

In [257]:
df_join = pd.merge(df_encabezado, df_items, on='id', how='inner')

#drop duplicates
df_join = df_join.drop_duplicates()
df_join 

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE
0,1,ZUANA BEACH RESORT,CRA 2 NO. 6-80 BELLOHORIZONTE,CRA 2 NO. 6-80 BELLOHORIZONTE,72035,2025-08-03,2025-08-11,PAQX35GR,DON CHICHARRON,192.0,UNIDAD,3166.19,607908.48
1,1,ZUANA BEACH RESORT,CRA 2 NO. 6-80 BELLOHORIZONTE,CRA 2 NO. 6-80 BELLOHORIZONTE,72035,2025-08-03,2025-08-11,UNDX50GR,ACHIRAS,24.0,UNIDAD,3761.90,90285.60
2,1,ZUANA BEACH RESORT,CRA 2 NO. 6-80 BELLOHORIZONTE,CRA 2 NO. 6-80 BELLOHORIZONTE,72035,2025-08-03,2025-08-11,PAQX100GR,DON CHICHARRON,100.0,UNIDAD,8565.00,856500.00


In [258]:
#Valor unidad y total, revisar trocado de valores 

In [259]:
#Bby ->Tostato
#Formatos Llenar manual 
#DIHEGO
#Merkeo
#Farmatodo 
#Panamericana
#Alkosto 
#Sapia -> .zip